In [6]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import csv
import tensorflow as tf
from tensorflow.keras.models import Model
from neurite.tf import models  # Assuming the module's location
import voxelmorph.tf.losses as vtml
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
import neurite as ne
import sys
import nibabel as nib
from tensorflow.keras.models import load_model
from neurite_sandbox.tf.models import labels_to_labels
import nibabel as nib
import tqdm
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import neurite as ne
import voxelmorph as vxm
from utils import *
import pathlib
import surfa as sf

    
mgh = pathlib.Path('fetus_label_map')
mgh_files = list(mgh.glob('*.nii.gz'))
label_maps = [np.uint8(sf.load_volume(str(file_path)).data) for file_path in mgh_files]



labels = np.unique(label_maps)
in_shape = label_maps[0].shape

gen_arg = {
    'in_shape': in_shape,
    'labels_in': labels_in,
    'labels_out': {f: 1 if f in (1, 2, 3, 4, 5, 6, 7) else 0 for f in labels_in},
    'one_hot':True,
    'aff_shift':0,
    'aff_scale':2,
    'up_scale':False,
    'aff_rotate':0,
    'aff_shear':0,
    'zero_background':0
}



#brain
model1 = ne.models.labels_to_image_new(
    in_shape=in_shape,
    labels_in=labels_in,
    labels_out={i: i if i in (1, 2, 3, 4, 5, 6, 7) else 0 for i in labels_in},
    warp_min=0.01,  # Adjust this value for a small warping change
    warp_max=0.02,  # Adjust this value for a small warping change
    aff_shift=10,
    one_hot=False,
    aff_rotate=180,
    aff_scale=2
)

#body
model2 = ne.models.labels_to_image_new(
    in_shape=in_shape,
    labels_in=labels_in,
    labels_out={i: 0 if i in (1, 2, 3, 4, 5, 6, 7) else i for i in labels_in},
    aff_shift=10,
    aff_rotate=5,
    aff_shear=0.05,
    blur_max=0.05,
    warp_min=0.01,  # Adjust this value for a small warping change
    warp_max=5,  # Adjust this value for a small warping change
    slice_prob=1,
    one_hot=False,
    crop_prob=1,
)

def gen_brain_fov(brain_maps, fov_maps):

    gen1 = generator3D_noshape(brain_maps)
    gen2 = generator3D_noshape(fov_maps)
    
    input_brain = next(gen1)
    input_fov = next(gen2)
    
    _ , output_brain = model1(input_brain)
    _ , output_fov = model2(input_fov)

    output_brain = np.squeeze(output_brain, axis=(0, -1))
    output_fov = np.squeeze(output_fov, axis=(0, -1))


    output_label = output_brain + output_fov * (output_brain == 0) 
    
    output_label = np.expand_dims(output_label, axis=0)  # Add a singleton dimension at the beginning
    output_label = np.expand_dims(output_label, axis=-1)
    return output_label



# unet_model = vxm.networks.Unet(inshape=(*in_shape, 1), nb_features=(en, de), 
#                                nb_conv_per_level=2,
#                                final_activation_function='softmax')

# labels_to_image_model = ne.models.labels_to_image_new(**gen_arg)


labels = np.unique(label_maps)
num_labels=8
labels_in = range(max(labels) + num_labels + 1)

brain_maps = get_brain(label_maps)
fov_maps = get_fov(label_maps)



gen = gen_brain_fov(brain_maps, fov_maps)


(None, 256, 256, 256, 1)
(None, 256, 256, 256, 1)
(None, 256, 256, 256, 1)
(None, 256, 256, 256, 1)


In [5]:
output_brain.shape

NameError: name 'output_brain' is not defined